** Image segmentation test using OpenCV and the watershed algorithm**

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt


In [ ]:
# Import image
img = cv2.imread('images/500um_2.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
plt.figure(figsize=(15,10))
plt.imshow(gray, cmap='gray')

** Detecting etched regions **

Otsu thresholding:

In [ ]:
# Thresholding
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

plt.figure
plt.hist(gray.ravel(),256)
plt.title("Intensity histogram")
plt.figure(figsize=(20,13))
plt.imshow(thresh, cmap='gray')
plt.title("Otsu algorithm")
plt.show()

In [ ]:
edge_row, edge_col = np.where(thresh == 255)
gray_edges = gray
gray_edges[edge_row, edge_col] = gray[1000,2400]
print(gray[1000,2400])
plt.figure(figsize=(20,13))
plt.imshow(gray_edges, cmap='gray')

In [ ]:
# 2nd pass with otsu algorithm
ret2, thresh2 = cv2.threshold(gray_edges,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

plt.figure
plt.hist(gray_edges.ravel(),256)
plt.title("Intensity histogram")
plt.figure(figsize=(20,13))
plt.imshow(thresh2, cmap='gray')

Filtering and edge detection 

In [ ]:
# Filter
f_kernel = np.array([[-1,-1,-1],
                     [-1,9,-1],
                     [-1, -1, -1]])
gray_f = cv2.filter2D(gray, -1, f_kernel)

plt.figure(figsize=(20,13))
plt.title("Grayscale filtered")
plt.imshow(gray_f, cmap='gray')

# Edge detection
edges = cv2.Canny(gray_f, 40, 90)

plt.figure(figsize=(20,13))
plt.imshow(edges, cmap='gray')
plt.show()

In [ ]:
# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(th3,cv2.MORPH_OPEN,kernel, iterations = 2)
# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)
# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)
plt.imshow(unknown, cmap='gray')

In [ ]:
# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
# Now, mark the region of unknown with zero
markers[unknown==255] = 0
plt.imshow(unknown, cmap='gray')

In [ ]:
markers = cv2.watershed(img,markers)
img[markers == -1] = [255,0,0]

In [ ]:
plt.imshow(img,cmap='gray')